# Лабораторная работа №1

## Задание 1

*Найти набор данных (датасет) для классификации удовлетворяющий следующим условиям: более 10 000 строк, более 20 столбцов, разные типы в столбцах, обязательно наличие целевого признака (таргета).*

Импортируем необходимые библиотеки.

In [2]:
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

Загружаем датасет и выбераем из него 50000 тыс. случайных записей.

In [4]:
dataset = pd.read_csv('Australia Rainfall.csv', delimiter=',').sample(50000)

Убераем столбцы, которые не содержат полезной информации, а также столбцы содержащие большое количество пустых значений. После этого убераем все записи, у которых есть хотя бы одно пустое значений.

In [7]:
X = dataset.drop(columns=['Evaporation', 'Sunshine', 'Date', 'Cloud9am', 'Cloud3pm'])
X.dropna(inplace=True)

Заменяем столбцы со строковыми значениями на соответствующие им целочисленные. Столбцы RainToday и RainTomorrow заменяем на bool-аналоги, так как они содержат всего 2 возможных значения.

In [11]:
locations = X['Location'].unique()
X['Location'] = X['Location'].map({value: i for i, value in enumerate(locations)})

X['RainToday'] = X['RainToday'].map({'No': False, 'Yes': True})
X['RainTomorrow'] = X['RainTomorrow'].map({'No': False, 'Yes': True})

wgds = X['WindGustDir'].unique()
X['WindGustDir'] = X['WindGustDir'].map({value: i for i, value in enumerate(wgds)})

wd9s = X['WindDir9am'].unique()
X['WindDir9am'] = X['WindDir9am'].map({value: i for i, value in enumerate(wd9s)})

wd9s = X['WindDir3pm'].unique()
X['WindDir3pm'] = X['WindDir3pm'].map({value: i for i, value in enumerate(wd9s)})
X

,Location,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
72885,0,16.5,34.3,0.0,0,30.0,0,0,19.0,13.0,56.0,20.0,1014.8,1011.4,19.9,32.3,False,False
9453,1,21.6,28.2,10.8,1,48.0,1,1,7.0,19.0,79.0,68.0,1013.7,1012.6,26.1,27.1,True,True
90284,2,21.6,28.5,0.0,0,41.0,1,2,26.0,24.0,60.0,51.0,1017.4,1015.9,25.1,27.5,False,False
132999,3,9.5,16.3,0.0,2,46.0,2,3,35.0,9.0,70.0,60.0,1012.7,1008.4,11.7,15.9,False,False
79577,4,9.3,15.1,1.0,3,69.0,3,3,17.0,13.0,77.0,69.0,1012.8,1008.8,11.9,13.6,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143040,42,23.3,36.4,0.4,7,35.0,5,11,2.0,6.0,81.0,42.0,1007.8,1004.9,27.4,35.2,False,False
111965,8,14.4,23.5,0.0,0,50.0,0,0,26.0,24.0,58.0,51.0,1014.5,1012.1,20.3,22.1,False,False
11555,1,25.5,34.9,0.0,9,57.0,3,12,19.0,37.0,58.0,66.0,1006.4,1003.2,29.9,29.5,False,False
96189,28,24.8,33.9,1.8,1,37.0,6,1,17.0,13.0,76.0,72.0,1010.8,1007.4,29.0,29.8,True,True


Отделяем таргетный столбец RainTomorrow от остального датасета.

In [14]:
y = X['RainTomorrow']
X = X.drop('RainTomorrow', axis=1)

Нормализуем данные в пределах от 0 до 1.

In [17]:
scaler = MinMaxScaler()
X_ans = scaler.fit_transform(X)

Разделяем датасет на тествую и тренировочную части в отношении 70/30.

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X_ans, y, test_size=0.3)

## Задание 2

*Провести классификацию найденного датасета, методом к-ближайших соседей. В формате Markdown писать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.*

Импортируем класс KNeighborsClassifier для классификации датасета методом k-ближайших соседей.

In [25]:
from sklearn.neighbors import KNeighborsClassifier

Создаём объект класса KNeighborsClassifier, реализующий алгоритм k-ближайших соседей.

In [28]:
kn_classifier = KNeighborsClassifier()

Определяем параметры, которые мы хотим оптимизировать для модели KNN.
* 'n_neighbors': range(1,30) - определяет количество ближайших соседей, используемых для классификации.
* 'metric': ['minkowski', 'euclidean', 'manhattan'] - определяет список метрик, которые мы хотим попробовать для измерения расстояния между точками данных.

In [31]:
kn_params ={
    'n_neighbors': range(1,30),
    'metric':['minkowski','euclidean','manhattan']  
}

Создаём объект класса GridSearchCV, выполняющий поиск оптимальных параметров модели, перебирая все возможные комбинации значений параметров.
* cv=5 - определяет количество фолдов (разделений) для кросс-валидации. В данном случае 5-кратная кросс-валидация.
* n_jobs=-1 - определяет количество ядер процессора, которые будут использоваться для параллельного выполнения Grid Search.

In [34]:
kn_grid = GridSearchCV(kn_classifier, kn_params, cv=5, n_jobs=-1)
kn_grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'metric': ['minkowski', 'euclidean', 'manhattan'],
                         'n_neighbors': range(1, 30)})

Обучаем модель на тренировочных данных.

In [36]:
kn_best_model = KNeighborsClassifier(**kn_grid.best_params_)
kn_best_model.fit(X_train, y_train)

KNeighborsClassifier(metric='manhattan', n_neighbors=21)

Применяем обученную модель к тестовым данным X_test и делаем предсказания.

In [38]:
kn_predicted = kn_best_model.predict(X_test)

Выводим отчёт.

In [40]:
print('Параметры:', kn_grid.best_params_)
print('Оценка:\n', metrics.classification_report(y_test, kn_predicted, digits=5))

Параметры: {'metric': 'manhattan', 'n_neighbors': 21}
Оценка:
               precision    recall  f1-score   support

       False    0.84771   0.96691   0.90340      9067
        True    0.77011   0.38953   0.51737      2580

    accuracy                        0.83901     11647
   macro avg    0.80891   0.67822   0.71038     11647
weighted avg    0.83052   0.83901   0.81789     11647

